In [ ]:
#-------------------------| BCC [100] Lithium slabs |--------------------------------#
#-----------------------------------|QE|---------------------------------------------#
from pathlib import Path
from ase.io import write
from ase.build.surface import bcc110

input_data = {
    'control': {
        'calculation': 'relax',
        'verbosity': 'high',
        'restart_mode': 'from_scratch',
        'nstep': 999,
        'tstress': False,
        'tprnfor': True,
        'outdir': './',
        'prefix': 'Li',
        'max_seconds': 86100,
        'etot_conv_thr': 1.0e-8,
        'forc_conv_thr': 1.0e-5,
        'disk_io': 'none',
        'pseudo_dir': './',
        'trism': False ,
    },
    'system': {
        'ibrav': 0,
        'tot_charge': 0.0,
        'tot_magnetization': -10000,
        'ecutwfc': 40.0,
        'ecutrho': 40*8,
        'occupations': 'smearing',
        'degauss': 0.01,
        'smearing': 'cold',
        # 'input_dft': 'rpbe',
        'nspin': 1,
        'assume_isolated': 'none',
        'esm_bc': 'pbc',
        'vdw_corr': 'none',
    },
    'electrons': {
        'electron_maxstep': 999,
        'scf_must_converge': True ,
        'conv_thr': 1.0e-11,
        'mixing_mode': 'local-TF',
        'mixing_beta': 0.6,
        'mixing_ndim': 8,
        'diagonalization': 'david',
        'diago_david_ndim': 2,
        'diago_rmm_ndim': 4,
        'diago_rmm_conv': False ,
        'diago_full_acc': False ,
    },
    'ions': {
        'ion_dynamics': 'bfgs',
        'upscale': 100,
        'bfgs_ndim': 6,
    },
    'cell': {
        'cell_dynamics': 'bfgs',
        'press_conv_thr': 0.1,
        'cell_dofree': 'all',
    }
}
pseudopotentials = {'Li': 'li_pbe_v1.4.uspp.F.UPF'}
base_directory_path = Path("/work/e89/e89/ba3g18/Working_dir/Applied-bias/Canonical-slabs/[110]")
for layer in range(1, 17, 2):
    Li_slab = bcc110('Li', (4, 6, layer), vacuum=7.5, a=3.439709869, orthogonal=True)
    Li_slab.set_cell([13.758839476, 14.59345304210515, 53.915874778947064])
    directory_name = f"{layer}-thick"
    full_directory = base_directory_path / directory_name
    full_directory.mkdir(parents=True, exist_ok=True)
    filename = full_directory / f'Li.in'
    write(filename, Li_slab, format='espresso-in', input_data=input_data, kpts=[7, 7, 1], pseudopotentials=pseudopotentials, crystal_coordinates=True)


In [ ]:
#-------------------------| BCC [100] Lithium slabs |--------------------------------#
#-----------------------------------|ONETEP|-----------------------------------------#

from pathlib import Path
from ase.io import write
from ase.build.surface import bcc111, bcc100, bcc110
from ase.io.onetep import get_onetep_keywords

base_directory_path = Path("/work/e89/e89/ba3g18/Working_dir/Applied-bias/Canonical-slabs/[100]")
keywords = get_onetep_keywords('/work/e89/e89/ba3g18/Working_dir/Keywords/Minimal.dat')
for layer in range(1, 15+1, 2):
    Li_slab = bcc100('Li', (6, 6, layer), a=3.4310707, vacuum=30, orthogonal=True)
    Li_slab.set_cell([20.5864242, 20.5864242, 84.0174949])
    Li_slab.set_tags([0] * len(Li_slab))
    directory_name = f"{layer}-thick"
    full_directory = base_directory_path / directory_name
    full_directory.mkdir(parents=True, exist_ok=True)
    filename = full_directory / f'Li.dat'
    write(filename, Li_slab, format='onetep-in', keywords=keywords)


In [ ]:
#-------------------------|PZC Lithium slabs w.r.t Li+/Li|---------------------------#
#-----------------------------------|ONETEP|-----------------------------------------#


from pathlib import Path
from ase.io import write
from ase.build.surface import bcc100
from ase.io.onetep import get_onetep_keywords

edft_electrode_potentials = {1: -3.294561248, 3: -2.906652439, 5: -2.784891632 , 7: -2.722812856 , 9: -2.641704425 , 11: -2.593243493 , 13: -2.563838481 , 15: -2.531885811, 17: -2.49595954}
edft_reference_potential = -4.30 
base_directory_path = Path("/work/e89/e89/ba3g18/Working_dir/Tester")
keywords = get_onetep_keywords('/work/e89/e89/ba3g18/Working_dir/Keywords/Grand-canonical.dat')
for layer in range(1, 17+1, 2):
    Li_slab = bcc100('Li', (4, 4, layer), a=3.41466082437722, vacuum=30.0)
    Li_slab.set_cell([13.76217048717384, 13.76217048717384, 85.00000000])
    Li_slab.set_tags([0] * len(Li_slab))
    edft_potential = edft_electrode_potentials.get(layer, 0.0)
    applied_value = edft_reference_potential - edft_potential
    keywords['edft_electrode_potential'] = f"{applied_value + 0.0} V"
    directory_name = f"{layer}-thick"
    full_directory = base_directory_path / directory_name
    full_directory.mkdir(parents=True, exist_ok=True)
    filename = full_directory / f'Li.dat'
    write(filename, Li_slab, format='onetep-in', keywords=keywords)


In [ ]:
#--------------------------|Lithium slabs with N2 absorbed|--------------------------#
#-----------------------------------|ONETEP|-----------------------------------------#
from ase import Atoms
from pathlib import Path
from ase.io import write
from ase.io.onetep import get_onetep_keywords
from ase.build.surface import bcc100, add_adsorbate

base_directory_path = Path("/work/e89/e89/ba3g18/Working_dir/tester")
keywords = get_onetep_keywords('/work/e89/e89/ba3g18/Working_dir/Keywords.dat')
Li_slab = bcc100('Li', (3, 3, 10), a=3.44, vacuum=25)
Li_slab.set_tags([0]*len(Li_slab))

molecule = Atoms('2N', positions=[(0.0, 0.0, 0.0), (0.0, 0.0, 1.1)])
molecule.center()
distances = [1.2, 1.4, 1.6, 1.8,  2.0 , 2.2 , 2.4]

for distance in distances:
    slab_copy = Li_slab.copy() 
    add_adsorbate(slab_copy, molecule, distance, 'ontop')  
    directory_name = f"{distance}_ang"
    full_directory = base_directory_path / directory_name
    full_directory.mkdir(parents=True, exist_ok=True)
    filename = full_directory / f'Li-{distance}.dat'
    write(filename, slab_copy, format='onetep-in', keywords=keywords)  


In [ ]:
#---------------------------|Lithium slabs N-2 layer fixed|--------------------------#
#--------------------------------------|ONETEP|--------------------------------------#
from ase.build import bcc100
from ase.visualize import view
from ase.io.onetep import get_onetep_keywords

keywords_1 = get_onetep_keywords('keywords_1.dat')
keywords_2 = get_onetep_keywords('keywords_2.dat')
for layer in range(1, 17+1, 2):
    atoms = bcc100('Li', (4, 4, layer), a=3.466, vacuum=15)
    if layer >= 7:
        tags = atoms.get_tags()
        max_tag = max(tags)
        min_tag = min(tags)
        top_two_layers = set([max_tag - 1, max_tag])
        bottom_two_layers = set([min_tag, min_tag + 1])
        tags = [1 if tag in top_two_layers or tag in bottom_two_layers else 2 for tag in tags]
        atoms.set_tags(tags)
        write('Li_%s.dat' % layer, atoms, format='onetep-in', keywords=keywords_1)
    else:
        tags = [0] * len(atoms)
        atoms.set_tags(tags)
        write('Li_%s.dat' % layer, atoms, format='onetep-in', keywords=keywords_2)
   